In [1]:
import btrack
import napari
print("Napari version no.:", napari.__version__)
# make sure to use v0.4.2
import os,re,glob
import numpy as np
import tifffile as tiff
from tqdm import tqdm

Napari version no.: 0.4.2


# writing my own input data paths 

In [2]:
expt_list = ['GV0794/Pos4',
 'GV0794/Pos7',
 'GV0794/Pos16',
 'GV0795/Pos4',
 'GV0795/Pos7',
 'GV0795/Pos17',
 'GV0796/Pos4',
 'GV0796/Pos7',
 'GV0796/Pos21',
 'GV0796/Pos22',
 'GV0797/Pos4',
 'GV0797/Pos7',
 'GV0797/Pos20',
 'GV0797/Pos21',
 'GV0798/Pos4',
 'GV0798/Pos7',
 'GV0798/Pos20',
 'GV0798/Pos21',
 'GV0800/Pos4',
 'GV0800/Pos7',
 'GV0800/Pos16',
 'GV0801/Pos4',
 'GV0802/Pos4',
 'GV0804/Pos4',
 'GV0805/Pos4',
 'GV0806/Pos4',
 'GV0807/Pos4']

In [3]:
write_folder = '/media/nathan/Transcend/Nathan/data/h2b/apoptosis_information'
#tif_folder = '/media/nathan/Transcend/Nathan/data/h2b/{}/{}/stacks'.format(GV,pos,ch) ## need to put in for loop before calling Open_Napari

In [4]:
def highlight_apoptoses(viewer, tracks):
    apops = [tuple((track.t[-1], track.x[-1], track.y[-1])) for track in tracks if track.fate.name == 'APOPTOSIS']
    [print('apop ID, final frame', track.ID, track.t[-1]) for track in tracks if track.fate.name == 'APOPTOSIS']
    radius = 60
    duration = 10
    highlight_apops = [tuple(((apops[i][0]+t, apops[i][1]-radius, apops[i][2]-radius), 
                                   (apops[i][0]+t, apops[i][1]+radius, apops[i][2]-radius), 
                                   (apops[i][0]+t, apops[i][1]+radius, apops[i][2]+radius), 
                                   (apops[i][0]+t, apops[i][1]-radius, apops[i][2]+radius))) 
                                    for t in range(0, duration)
                                    for i in range(len(apops))]
    
    return viewer.add_shapes(highlight_apops, opacity = 0.4, shape_type = 'ellipse', face_color = 'transparent',edge_color = '#39FF14', edge_width = 5, name = 'Apoptosis')   

def Open_Napari(GV,pos,ch):

    # Load into Napari

    img_file = os.path.join(tif_folder,'{}_{}_{}_8_bit.tif'.format(GV,pos,ch))
    if os.path.exists(img_file) == False:
        return
    full_mov = tiff.imread(img_file)

    obj_type = 'obj_type_1' if ch == 'GFP' else 'obj_type_2'
    #HDF5_file = f'/mnt/lowe-sn00/Data/Giulia/{GV}/{pos}/{pos}_aligned/HDF/segmented.hdf5'
    HDF5_file = f'/media/nathan/Transcend/Nathan/data/h2b/{GV}/{pos}/{pos}_aligned/HDF/segmented.hdf5'

    with btrack.dataio.HDF5FileHandler(HDF5_file, 'r',obj_type=obj_type) as h:
        #segmentation = h.segmentation
        tracks = h.tracks
    
    # Re-format the tracks to be viewable alongside the image stacks
    data, properties, graph = btrack.utils.tracks_to_napari(tracks, ndim=2)
    x_data = np.array(data[:,3])
    y_data = np.array(data[:,2])
    data[:,2] = x_data
    data[:,3] = y_data

    # Open Napari

    metadata = {'img_file_name':img_file,
               'segmentation_file_name':HDF5_file}

    global selected_track_lengths
    global latest_added_track
    selected_track_lengths = []
    latest_added_track = [] # 's' if a stitched track, else 'r'

    txt_address = os.path.join(write_folder,f'{GV}_{pos}_{ch}_apoptoses.txt')

    # still produces an empty txt file if there are no apoptoses present
    txt_file = open(txt_address,'a')
    txt_file.close()

    with napari.gui_qt():

        SCALE = [1.0,1.0,1.0]
        viewer = napari.Viewer()
        viewer.add_image(full_mov,rgb=False)
        #viewer.add_image(segmentation,rgb=False,blending='additive')
        tracker_layer = viewer.add_tracks(data, properties=properties,graph=graph,name='tracks_1')
        points_layer = viewer.add_points(np.empty((0,3)),face_color='orange')
        stitched_points_layer = viewer.add_points(np.empty((0,3)),face_color='cyan')
#        shapes_layer = viewer.add_shapes(highlight_apops)
        highlight_apoptoses(viewer, tracks)

        @viewer.bind_key('r')
        def write_apoptosis(event=None):
            cell_ID = tracker_layer._get_value()
            position = tracker_layer.coordinates #.position
            cell_track = [track for track in tracks if track.ID == cell_ID][0]
            cell_positions = np.transpose(np.stack([cell_track.t,cell_track.x,cell_track.y],axis=0)) # (1,frames,3)
            points_layer.data = np.append(points_layer.data,cell_positions,axis=0)
            txt_entry = f'{GV}_{pos}_{cell_ID}_{ch} {int(position[0])}\n'
            with open(txt_address,'a') as txt_file:
                txt_file.write(txt_entry)
            txt_file.close()
            selected_track_lengths.append(len(cell_track.t))
            latest_added_track.append('r')

        @viewer.bind_key('s')
        def write_stitched_track(event=None):
            cell_ID = tracker_layer._get_value()
            position = tracker_layer.position
            cell_track = [track for track in tracks if track.ID == cell_ID][0]
            cell_positions = np.transpose(np.stack([cell_track.t,cell_track.x,cell_track.y],axis=0)) # (1,frames,3)
            stitched_points_layer.data = np.append(stitched_points_layer.data,cell_positions,axis=0)
            txt_entry = f'{GV}_{pos}_{cell_ID}_{ch} {int(position[0])} stitched\n'
            with open(txt_address,'a') as txt_file:
                txt_file.write(txt_entry)
            txt_file.close()
            selected_track_lengths.append(len(cell_track.t))
            latest_added_track.append('s')
            
#         @viewer.bind_key('') ## add pre-stitched function here?
#         def write_stitched_track(event=None):
#             cell_ID = tracker_layer._get_value()
#             position = tracker_layer.position
#             cell_track = [track for track in tracks if track.ID == cell_ID][0]
#             cell_positions = np.transpose(np.stack([cell_track.t,cell_track.x,cell_track.y],axis=0)) # (1,frames,3)
#             stitched_points_layer.data = np.append(stitched_points_layer.data,cell_positions,axis=0)
#             txt_entry = f'{GV}_{pos}_{cell_ID}_{ch} {int(position[0])} stitched\n'
#             with open(txt_address,'a') as txt_file:
#                 txt_file.write(txt_entry)
#             txt_file.close()
#             selected_track_lengths.append(len(cell_track.t))
#             latest_added_track.append('s')

        @viewer.bind_key('f')
        def write_fragment_track(event=None):
            cell_ID = tracker_layer._get_value()
            position = tracker_layer.position
            cell_track = [track for track in tracks if track.ID == cell_ID][0]
            cell_positions = np.transpose(np.stack([cell_track.t,cell_track.x,cell_track.y],axis=0)) # (1,frames,3)
            fragment_points_layer.data = np.append(fragment_points_layer.data,cell_positions,axis=0)
            selected_track_lengths.append(len(cell_track.t))
            latest_added_track.append('f')
        
        @viewer.bind_key('w')
        def erase_latest_entry(event=None):
            global selected_track_lengths 
            global latest_added_track
            with open(txt_address,'r+') as txt_file:
                lines = txt_file.readlines()
                txt_file.seek(0)
                for line in lines[:-1]:
                    txt_file.write(line)
                txt_file.truncate()
            txt_file.close()
            if latest_added_track[-1] == 's':
                stitched_points_layer.data = stitched_points_layer.data[:-selected_track_lengths[-1]]
            elif latest_added_track[-1] == 'r':
                points_layer.data = points_layer.data[:-selected_track_lengths[-1]]
            selected_track_lengths = selected_track_lengths[:-1]
            latest_added_track = latest_added_track[:-1]

In [ ]:
for line in expt_list:
    GV,pos = line.split('/')
    pos = re.sub('\n','',pos) # remove linespace
    for ch in ['GFP','RFP']:
        tif_folder = '/media/nathan/Transcend/Nathan/data/h2b/{}/{}/stacks'.format(GV,pos,ch)
        print(expt_list.index(line),GV,pos,ch)
        Open_Napari(GV,pos,ch)

0 GV0794 Pos4 GFP


[INFO][2021/03/05 04:30:30 PM] Opening HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/03/05 04:30:30 PM] Loading tracks/obj_type_1
[INFO][2021/03/05 04:30:31 PM] Loading objects/obj_type_1 (69360, 5) (64986 filtered: area>=100)
[INFO][2021/03/05 04:30:31 PM] Closing HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5


apop ID, final frame 12 955
apop ID, final frame 118 361
apop ID, final frame 393 861
0 GV0794 Pos4 RFP


[INFO][2021/03/05 04:31:07 pm] Opening HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5...
[INFO][2021/03/05 04:31:07 pm] Loading tracks/obj_type_2
[INFO][2021/03/05 04:31:07 pm] Loading objects/obj_type_2 (13865, 5) (10538 filtered: area>=100)
[INFO][2021/03/05 04:31:07 pm] Closing HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0794/Pos4/Pos4_aligned/HDF/segmented.hdf5


apop ID, final frame 6 665
apop ID, final frame 70 828
apop ID, final frame 73 831
apop ID, final frame 81 689
apop ID, final frame 84 697
apop ID, final frame 85 704
apop ID, final frame 86 709
apop ID, final frame 87 720
apop ID, final frame 88 785
apop ID, final frame 96 778
apop ID, final frame 98 795
apop ID, final frame 99 806
apop ID, final frame 101 811
apop ID, final frame 102 839
apop ID, final frame 104 823
apop ID, final frame 109 872
apop ID, final frame 112 870
apop ID, final frame 114 850
apop ID, final frame 130 915
apop ID, final frame 140 955
1 GV0794 Pos7 GFP


...

apop ID, final frame 29 320
apop ID, final frame 31 100
apop ID, final frame 36 142
apop ID, final frame 38 242
apop ID, final frame 39 25
apop ID, final frame 40 151
apop ID, final frame 2 41
apop ID, final frame 42 4
apop ID, final frame 44 10
apop ID, final frame 45 2
apop ID, final frame 48 152
apop ID, final frame 52 271
apop ID, final frame 4 1092
apop ID, final frame 7 4
apop ID, final frame 13 106
apop ID, final frame 24 35
apop ID, final frame 20 81
apop ID, final frame 55 17
apop ID, final frame 70 56
apop ID, final frame 73 20
apop ID, final frame 78 59
apop ID, final frame 77 68
apop ID, final frame 83 29
apop ID, final frame 88 69
apop ID, final frame 112 209
apop ID, final frame 113 65
apop ID, final frame 130 92
apop ID, final frame 162 261
apop ID, final frame 163 158
apop ID, final frame 168 187
apop ID, final frame 175 200
apop ID, final frame 178 202
apop ID, final frame 185 233
apop ID, final frame 192 227
apop ID, final frame 197 247
apop ID, final frame 198 266
ap

ERROR:root:Unhandled exception:
Traceback (most recent call last):
  File "/home/nathan/anaconda3/lib/python3.7/site-packages/vispy/app/backends/_qt.py", line 502, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/nathan/anaconda3/lib/python3.7/site-packages/vispy/app/backends/_qt.py", line 551, in _keyEvent
    func(native=ev, key=key, text=text_type(ev.text()), modifiers=mod)
  File "/home/nathan/anaconda3/lib/python3.7/site-packages/vispy/util/event.py", line 455, in __call__
    self._invoke_callback(cb, event)
  File "/home/nathan/anaconda3/lib/python3.7/site-packages/vispy/util/event.py", line 475, in _invoke_callback
    self, cb_event=(cb, event))
  File "/home/nathan/anaconda3/lib/python3.7/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    cb(event)
  File "/home/nathan/anaconda3/lib/python3.7/site-packages/napari/_qt/qt_viewer.py", line 698, in on_key_press
    self.viewer.press_key(combo)
  File "/home/nathan/anaco

20 GV0800 Pos16 GFP


[INFO][2021/03/08 10:48:49 pm] Opening HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0800/Pos16/Pos16_aligned/HDF/segmented.hdf5...
INFO:worker_process:Opening HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0800/Pos16/Pos16_aligned/HDF/segmented.hdf5...
[INFO][2021/03/08 10:48:49 pm] Loading tracks/obj_type_1
INFO:worker_process:Loading tracks/obj_type_1
[INFO][2021/03/08 10:48:52 pm] Loading objects/obj_type_1 (361372, 5) (345214 filtered: area>=100)
INFO:worker_process:Loading objects/obj_type_1 (361372, 5) (345214 filtered: area>=100)
[INFO][2021/03/08 10:48:55 pm] Closing HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0800/Pos16/Pos16_aligned/HDF/segmented.hdf5
INFO:worker_process:Closing HDF file: /media/nathan/Transcend/Nathan/data/h2b/GV0800/Pos16/Pos16_aligned/HDF/segmented.hdf5


apop ID, final frame 27 282
apop ID, final frame 52 5
apop ID, final frame 182 144
apop ID, final frame 392 410
apop ID, final frame 516 528
apop ID, final frame 697 638
apop ID, final frame 807 1143
apop ID, final frame 939 972
apop ID, final frame 996 1011
apop ID, final frame 1076 935
apop ID, final frame 1192 756
apop ID, final frame 1322 1084
apop ID, final frame 1400 820
apop ID, final frame 1470 1092
apop ID, final frame 1482 817
apop ID, final frame 1552 1190
apop ID, final frame 1607 863
apop ID, final frame 1766 1053
apop ID, final frame 1768 1018
apop ID, final frame 1842 887
apop ID, final frame 1845 883
apop ID, final frame 1873 949
apop ID, final frame 1872 1050
apop ID, final frame 2000 1035
apop ID, final frame 1999 1038
apop ID, final frame 2109 1020
apop ID, final frame 2106 959
apop ID, final frame 2447 1045
apop ID, final frame 2566 1019
apop ID, final frame 2932 1062
apop ID, final frame 3062 1080
apop ID, final frame 3100 1131
apop ID, final frame 3201 1155
apop I

# chris' misc

In [ ]:
# Import Experiment List

expt_list = list(open('/media/chris/Data/Harvested_Apoptoses/90_10_experiments.txt','r').readlines())

In [ ]:
expt_list

In [ ]:
write_folder = '/media/chris/Data/Harvested_Apoptoses'
tif_folder = '/media/chris/Data/Big_Tifs_Giulia_Movies'

In [ ]:
# Create Image Stacks

#GV,pos = 'GV0796','Pos4'

for line in expt_list:
    GV,pos = line.split('/')
    pos = re.sub('\n','',pos) # remove linespace
    pos_folder = f'/mnt/lowe-sn00/Data/Giulia/{GV}/{pos}/{pos}_aligned/'
    if os.path.exists(os.path.join(pos_folder,'tracks')) == False: # if there is no data in pos_folder
        continue
    for channel in [1,2]:
        # No. Timesteps
        ch = 'GFP' if channel == 1 else 'RFP'
        files = glob.glob(os.path.join(pos_folder,'img_channel00{}*.tif'.format(channel)))
        files.sort(key=lambda f:int(f[-18:-9]))
        images = []
        for tif_ad in tqdm(files):
            tif = np.array(tiff.imread(tif_ad),dtype=np.uint8)
            tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
            tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)]
            images.append(tif)
        images = np.stack(images,axis=0)
        with tiff.TiffWriter(os.path.join(tif_folder,'{}_{}_{}_8_bit.tif'.format(GV,pos,ch)),imagej=True) as tif_:
            tif_.save(images)